In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import defaultdict

import json
import pandas as pd

from scipy.stats import ttest_ind, wilcoxon, cramervonmises_2samp, brunnermunzel, epps_singleton_2samp, anderson_ksamp
from numpy import random

DEFAULT_RANDOM_SEED = 774
random.mtrand._rand.seed(DEFAULT_RANDOM_SEED)

In [4]:
category = "min_tpm_5"

In [6]:
extra_data_headers = pd.read_csv('../../data/extra_data.tsv', delimiter="\t", nrows=0).columns
data = pd.read_csv(f"../../preprocessed/{category}/genes.csv", delimiter=",", decimal='.')

subtypes = set(data["subtype"])
genes = set(data.columns) - set(extra_data_headers) - set(["prognostic"])

In [28]:
def select_important_genes_by_sex(metric, verify_zeros: bool = False):
  important_genes_by_sex_subtype = defaultdict(list)

  for sex in ["Male", "Female"]:
    filtered_sex_dataset = data[data["sex"] == sex]
    for subtype in subtypes:
      subtype_dataset = filtered_sex_dataset[filtered_sex_dataset["subtype"] == subtype]
      not_subtype_dataset = filtered_sex_dataset[filtered_sex_dataset["subtype"] != subtype]

      for gene in genes:
        sample_X = subtype_dataset[gene]
        sample_Y = not_subtype_dataset[gene]

        if verify_zeros:
          if len(sample_Y[sample_Y != 0]) == 0:
            if len(sample_X[sample_X != 0]) != 0:
              important_genes_by_sex_subtype[(sex, subtype)].append({ "gene": gene, "pvalue": 0 })  

            continue

        try:
          result = metric(sample_X, sample_Y)
          if (result.pvalue <= 0.001):
            important_genes_by_sex_subtype[(sex, subtype)].append({ "gene": gene, "pvalue": result.pvalue })
        except:
          continue

  result = defaultdict(dict)
  for key in important_genes_by_sex_subtype.keys():
    result[key[1]] |= { key[0]: sorted(important_genes_by_sex_subtype[key], key=lambda x: x["pvalue"]) }

  return result


In [24]:
def select_and_save(metric_name: str, metric, verify_zeros: bool = False):
  result = select_important_genes_by_sex(metric, verify_zeros)
  open(f"../../preprocessed/{category}/important_genes_{metric_name}_pvalue.json", "w").write(json.dumps(result))

In [ ]:
select_and_save("ttest", lambda x, y: ttest_ind(x, y, equal_var=False))

Total selected genes: 13120


In [ ]:
select_and_save("wilcoxon", lambda x, y: wilcoxon(x, y.sample(len(x)), zero_method="zsplit"))

/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_wilcoxon.py:199: UserWarning: Sample size too small for normal approximation.
  temp = _wilcoxon_iv(x, y, zero_method, correction, alternative, method, axis)


Total selected genes: 9912


In [ ]:
select_and_save("cramervonmises", cramervonmises_2samp)

Total selected genes: 13171


In [9]:
select_and_save("brunnermunzel", brunnermunzel)

/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:9171: RuntimeWarning: invalid value encountered in scalar divide
  wbfn /= (nx + ny) * np.sqrt(nx * Sx + ny * Sy)
/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:9177: RuntimeWarning: invalid value encountered in scalar divide
  df = df_numer / df_denom
/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:573: RuntimeWarning: p-value cannot be estimated with `distribution='t' because degrees of freedom parameter is undefined (0/0). Try using `distribution='normal'
  res = hypotest_fun_out(*samples, **kwds)
/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:9171: RuntimeWarning: divide by zero encountered in scalar divide
  wbfn /= (nx + 

In [29]:
select_and_save("epps_singleton", epps_singleton_2samp, verify_zeros=True)

/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_hypotests.py:120: RuntimeWarning: divide by zero encountered in divide
  ts = np.reshape(t, (-1, 1)) / sigma
/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_hypotests.py:123: RuntimeWarning: invalid value encountered in multiply
  gx = np.vstack((np.cos(ts*x), np.sin(ts*x))).T  # shape = (nx, 2*len(t))
/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_hypotests.py:123: RuntimeWarning: invalid value encountered in cos
  gx = np.vstack((np.cos(ts*x), np.sin(ts*x))).T  # shape = (nx, 2*len(t))
/Users/igor.mandello/unicamp/acute-lymphocytic-leukemia-analysis/.venv/lib/python3.12/site-packages/scipy/stats/_hypotests.py:123: RuntimeWarning: invalid value encountered in sin
  gx = np.vstack((np.cos(ts*x), np.sin(ts*x))).T  # shape = (nx, 2*len(t))
/Users/i

In [26]:
select_and_save("anderson_ksamp", lambda x, y: anderson_ksamp([x, y]), verify_zeros=True)

/var/folders/vp/tqkr07mj3cn274npl2jmvy5w0000gq/T/ipykernel_26096/678088710.py:1: UserWarning: p-value capped: true value larger than 0.25. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)
  select_and_save("anderson_ksamp", lambda x, y: anderson_ksamp([x, y]), verify_zeros=True)
/var/folders/vp/tqkr07mj3cn274npl2jmvy5w0000gq/T/ipykernel_26096/678088710.py:1: UserWarning: p-value floored: true value smaller than 0.001. Consider specifying `method` (e.g. `method=stats.PermutationMethod()`.)
  select_and_save("anderson_ksamp", lambda x, y: anderson_ksamp([x, y]), verify_zeros=True)


In [24]:
def select_important_genes_overall(metric):
  important_genes_by_subtype = defaultdict(list)

  for subtype in subtypes:
    subtype_dataset = data[data["subtype"] == subtype]
    not_subtype_dataset = data[data["subtype"] != subtype]

    for gene in genes:
      result = metric(subtype_dataset[gene], not_subtype_dataset[gene])
      if (result.pvalue <= 0.001):
        important_genes_by_subtype[subtype].append({ "gene": gene, "pvalue": result.pvalue })

  result = {}
  for key in important_genes_by_subtype.keys():
    result[key] = sorted(important_genes_by_subtype[key], key=lambda x: x["pvalue"])

  return result


In [25]:
ttest_result = select_important_genes_overall(lambda x, y: ttest_ind(x, y, equal_var=False))

In [26]:
open(f"../preprocessed/{category}/important_genes_ttest_overall_pvalue.json", "w").write(json.dumps(ttest_result))

2467220